4

In [10]:
import requests
import torch
from io import BytesIO
def get_s3_state_dict(s3_url):
    response = requests.get(s3_url, stream=True)
    bio = BytesIO(response.content)
    state_dict = torch.load(bio)
    return state_dict
#     model.load_state_dict(loaded["model"])

# s3_url = "https://uberduck-models.s3.us-west-1.amazonaws.com/radtts++ljs-dap.pt"
s3_url = "https://uberduck-models-us-west-2.s3.us-west-2.amazonaws.com/radtts%2B%2Bljs-dap.pt"
pretrained_dict = get_s3_state_dict(s3_url)['state_dict']


In [13]:
from uberduck_ml_dev.models.radtts import RADTTS
from uberduck_ml_dev.exec.train_radtts_with_ray import model_config

include_layers = []
ignore_layers_warmstart = []
# "warmstart"
model = RADTTS(
    **model_config,
)

if len(include_layers):
    pretrained_dict = {k: v for k, v in pretrained_dict.items()
                       if any(l in k for l in include_layers)}

if len(ignore_layers_warmstart):
    pretrained_dict = {k: v for k, v in pretrained_dict.items()
                       if all(l not in k for l in ignore_layers_warmstart)}

model_dict = model.state_dict()
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict, strict=False)
# print("Warm started from {}".format(checkpoint_path))
# model.load_state_dict(state_dict['state_dict'])

Applying spectral norm to text encoder LSTM
Applying spectral norm to context encoder LSTM


_IncompatibleKeys(missing_keys=[], unexpected_keys=['dur_pred_layer.bottleneck_layer.projection_fn.conv.bias', 'dur_pred_layer.bottleneck_layer.projection_fn.conv.weight_g', 'dur_pred_layer.bottleneck_layer.projection_fn.conv.weight_v', 'dur_pred_layer.feat_pred_fn.convolutions.0.bias', 'dur_pred_layer.feat_pred_fn.convolutions.0.weight_g', 'dur_pred_layer.feat_pred_fn.convolutions.0.weight_v', 'dur_pred_layer.feat_pred_fn.convolutions.1.bias', 'dur_pred_layer.feat_pred_fn.convolutions.1.weight_g', 'dur_pred_layer.feat_pred_fn.convolutions.1.weight_v', 'dur_pred_layer.feat_pred_fn.bilstm.weight_ih_l0', 'dur_pred_layer.feat_pred_fn.bilstm.bias_ih_l0', 'dur_pred_layer.feat_pred_fn.bilstm.bias_hh_l0', 'dur_pred_layer.feat_pred_fn.bilstm.weight_ih_l0_reverse', 'dur_pred_layer.feat_pred_fn.bilstm.bias_ih_l0_reverse', 'dur_pred_layer.feat_pred_fn.bilstm.bias_hh_l0_reverse', 'dur_pred_layer.feat_pred_fn.bilstm.weight_hh_l0_orig', 'dur_pred_layer.feat_pred_fn.bilstm.weight_hh_l0_reverse_orig',

In [15]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_ray_dataset
ray_dataset = get_ray_dataset()
datum = ray_dataset.take(1)

2023-03-08 14:52:12,052	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.38.167:6379...
2023-03-08 14:52:12,059	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-08 14:52:12,078	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_9ec71b75650aa97c22e1aaf3f034b3cf.zip' (7.00MiB) to Ray cluster...
2023-03-08 14:52:12,159	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_9ec71b75650aa97c22e1aaf3f034b3cf.zip'.
(_get_read_tasks pid=40462) 2023-03-08 14:52:13,735	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.
2023-03-08 14:52:14,981	INFO bulk_executor.py:41 -- Executing DAG InputData

(autoscaler +15m23s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +15m23s) Adding 2 node(s) of type worker-node-type-0.


2023-03-08 14:52:26,517	WARNING worker.py:1870 -- WARNING: 20 PYTHON worker processes have been started on node: 0d525869aa4fbe7486608cd1f978cbcab70a1e9b9f62ca8f1ab55815 with address: 10.0.38.167. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
2023-03-08 14:52:26,981	WARNING worker.py:1870 -- WARNING: 24 PYTHON worker processes have been started on node: 0d525869aa4fbe7486608cd1f978cbcab70a1e9b9f62ca8f1ab55815 with address: 10.0.38.167. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
MapBatches(<lambda>): 100%|██████████| 100/100 [00:00<00:00, 390.35it/s]
2023-03-08 14:52:34,218	WARNING plan.py:528 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more

(autoscaler +15m38s) Adding 2 node(s) of type worker-node-type-0.


In [18]:
from ray.air import session
dataset_shard = session.get_dataset_shard("train")

NameError: name 'session' is not defined

(autoscaler +21m25s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +21m30s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +21m35s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +21m40s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +21m45s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +21m50s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +21m55s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m0s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m5s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m9s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m15s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m20s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m25s) Removing 4 nodes of type worker-node-type-0 (idle).
(autoscaler +22m30s) Removing 4 nodes of type worker-n

In [17]:
# datum
batch = next(enumerate(
            dataset_shard.iter_batches(batch_size=batch_size)
        ))

(raylet, ip=10.0.35.179) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.35.179)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.19.249) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.19.249)   aiogrpc.init_grpc_aio()


(autoscaler +16m23s) Resized to 36 CPUs, 1 GPUs.


(raylet, ip=10.0.48.237) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.48.237)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.28.173) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.28.173)   aiogrpc.init_grpc_aio()


(autoscaler +16m38s) Resized to 68 CPUs, 1 GPUs.


In [ ]:
batch_dict = ray_df_to_batch_radtts(batch)
mel = to_gpu(batch_dict['mel'])
speaker_ids = to_gpu(batch_dict['speaker_ids'])
attn_prior = to_gpu(batch_dict['attn_prior'])
f0 = to_gpu(batch_dict['f0'])
voiced_mask = to_gpu(batch_dict['voiced_mask'])
p_voiced = to_gpu(batch_dict['p_voiced'])
text = to_gpu(batch_dict['text'])
in_lens = to_gpu(batch_dict['input_lengths'])
out_lens = to_gpu(batch_dict['output_lengths'])
energy_avg = to_gpu(batch_dict['energy_avg'])

outputs = model(
            mel, speaker_ids, text, in_lens, out_lens,
            binarize_attention=binarize, attn_prior=attn_prior,
            f0=f0, energy_avg=energy_avg,
            voiced_mask=voiced_mask, p_voiced=p_voiced)

In [1]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_ray_dataset, train_func, train_config, model_config
from ray.train.torch import TorchTrainer, TorchCheckpoint, TorchTrainer
from ray.air.config import ScalingConfig, RunConfig
from ray.tune import SyncConfig

class TorchCheckpointFixed(TorchCheckpoint):
    def __setstate__(self, state: dict):
        if "_data_dict" in state and state["_data_dict"]:
            state = state.copy()
            state["_data_dict"] = self._decode_data_dict(state["_data_dict"])
        super(TorchCheckpoint, self).__setstate__(state)


# result = trainer.fit()

In [2]:
from ray.air.integrations.wandb import  setup_wandb

In [3]:
ray_dataset = get_ray_dataset()

2023-03-08 14:27:33,779	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.38.167:6379...
2023-03-08 14:27:33,828	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-08 14:27:34,150	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_78efb00b2d4bf87cf2071ad4d319358d.zip' (7.03MiB) to Ray cluster...
2023-03-08 14:27:34,245	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_78efb00b2d4bf87cf2071ad4d319358d.zip'.
(_get_read_tasks pid=25299) 2023-03-08 14:27:35,903	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.


In [4]:

train_config['n_group_size'] = model_config['n_group_size']
train_config['dur_model_config'] = model_config['dur_model_config']
train_config['f0_model_config'] = model_config['f0_model_config']
train_config['energy_model_config'] = model_config['energy_model_config']
train_config['v_model_config']=model_config['v_model_config']

trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config=train_config,
    scaling_config=ScalingConfig(
        num_workers=2, use_gpu=True, resources_per_worker=dict(CPU=4, GPU=1)
    ),
    run_config=RunConfig(
        sync_config=SyncConfig(upload_dir="s3://uberduck-anyscale-data/checkpoints")
    ),
    datasets={"train": ray_dataset},
)


In [5]:
config = train_config
setup_wandb(config, project="radtts-ray", rank_zero_only=False)

2023-03-07 16:35:52,269	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_trial_id` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
2023-03-07 16:35:52,271	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_trial_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
2023-03-07 16:35:52,272	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_experiment_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
wandb: Currently logged in as: uberduck. Use `wandb 

In [8]:
import torch

print("CUDA AVAILABLE: ", torch.cuda.is_available())
epochs = config["epochs"]
batch_size = config["batch_size"]
steps_per_sample = config["steps_per_sample"]
# gin_channels = config["gin_channels"]
sigma = config['sigma']
kl_loss_start_iter = config['kl_loss_start_iter']
binarization_start_iter = config['binarization_start_iter']


CUDA AVAILABLE:  True


NameError: name 'RADTTS' is not defined

In [5]:

# import ray.train as train
# model = train.torch.prepare_model(model, parallel_strategy_kwargs = dict(find_unused_parameters=True))

Applying spectral norm to text encoder LSTM
Applying spectral norm to context encoder LSTM


/home/ray/default/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered 

NameError: name 'state_dict' is not defined

In [ ]:


# discriminator = MultiPeriodDiscriminator(MODEL_CONFIG["use_spectral_norm"])
# discriminator = train.torch.prepare_model(discriminator)

checkpoint_dict = _load_checkpoint_dict()
if checkpoint_dict is None:
    global_step = 0
    start_epoch = 0
else:
    global_step = checkpoint_dict["global_step"]
    start_epoch = checkpoint_dict["epoch"]
    if session.get_world_size() > 1:
        model_sd = checkpoint_dict["model"]
        # NOTE(zach): Add audio embedding state dict if it is not present.
        # NOTE(zach): Pass strict=False due to different nuber of gin_channels
        model.load_state_dict(model_sd, strict=False)
    else:
        model_sd = _fix_state_dict(checkpoint_dict["model"])
        # NOTE(zach): Pass strict=False due to different nuber of gin_channels
        model.load_state_dict(model_sd, strict=False)
    del checkpoint_dict

# NOTE (Sam): replace with RAdam
optim = torch.optim.Adam(
    model.parameters(),
    lr = config["learning_rate"],
    weight_decay = config["weight_decay"]
)
scheduler = ExponentialLR(
    optim,
    config["weight_decay"],
    last_epoch=-1,
)
dataset_shard = session.get_dataset_shard("train")
global_step = 0
scaler = GradScaler()

criterion = RADTTSLoss(
    sigma,
    config['n_group_size'],
    config['dur_model_config'],
    config['f0_model_config'],
    config['energy_model_config'],
    vpred_model_config=config['v_model_config'],
    loss_weights=config['loss_weights']
)
attention_kl_loss = AttentionBinarizationLoss()
iteration = 0
for epoch in range(start_epoch, start_epoch + epochs):
    for batch_idx, ray_batch_df in enumerate(
        dataset_shard.iter_batches(batch_size=batch_size)
    ):
        torch.cuda.empty_cache()
        _train_step(
            ray_batch_df,
            model,
            optim,
            global_step,
            steps_per_sample,
            scaler,
            scheduler,
            criterion,
            attention_kl_loss,
            iteration,
            kl_loss_start_iter,
            binarization_start_iter,
        )
        global_step += 1
    if session.get_world_rank() == 0:
        # TODO(zach): Also save wandb artifact here.
        checkpoint = Checkpoint.from_dict(
            dict(
                epoch=epoch,
                global_step=global_step,
                model=model.state_dict(),
            )
        )
        session.report({}, checkpoint=checkpoint)
        artifact = wandb.Artifact(
            f"artifact_epoch{epoch}_step{global_step}", "model"
        )
        with tempfile.TemporaryDirectory() as tempdirname:
            checkpoint.to_directory(tempdirname)
            artifact.add_dir(tempdirname)
            wandb.log_artifact(artifact)
        iteration += 1

2023-03-07 16:19:08,608	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-07 16:19:08,633	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-07 16:19:08,653	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9.zip' (6.85MiB) to Ray cluster...
2023-03-07 16:19:08,744	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9.zip'.
(_get_read_tasks pid=5467, ip=10.0.46.245) 2023-03-07 16:19:10,287	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.


2023-03-07 16:19:11,683	WARNING trial_runner.py:1333 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (202 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(RayTrainWorker pid=235, ip=10.0.45.178) 2023-03-07 16:19:23,587	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:19:24,250	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:19:27,668	INFO bulk_e

(RayTrainWorker pid=417, ip=10.0.17.31) CUDA AVAILABLE:  True
(RayTrainWorker pid=417, ip=10.0.17.31) Applying spectral norm to text encoder LSTM
(RayTrainWorker pid=417, ip=10.0.17.31) Applying spectral norm to context encoder LSTM


(RayTrainWorker pid=417, ip=10.0.17.31) /tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/working_dir_files/_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
(RayTrainWorker pid=417, ip=10.0.17.31) The boolean parameter 'some' has been replaced with a string parameter 'mode'.
(RayTrainWorker pid=417, ip=10.0.17.31) Q, R = torch.qr(A, some)
(RayTrainWorker pid=417, ip=10.0.17.31) should be replaced with
(RayTrainWorker pid=417, ip=10.0.17.31) Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
(RayTrainWorker pid=417, ip=10.0.17.31)   W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor o

(RayTrainWorker pid=235, ip=10.0.45.178) CUDA AVAILABLE:  True
(RayTrainWorker pid=235, ip=10.0.45.178) Applying spectral norm to text encoder LSTM


...

(RayTrainWorker pid=235, ip=10.0.45.178) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=235, ip=10.0.45.178)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=417, ip=10.0.17.31)   result = 

(RayTrainWorker pid=417, ip=10.0.17.31) Loss: 5.8561625480651855
(RayTrainWorker pid=235, ip=10.0.45.178) Loss: 5.837704658508301


(RayTrainWorker pid=235, ip=10.0.45.178) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=235, ip=10.0.45.178)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=417, ip=10.0.17.31)   result = 

(RayTrainWorker pid=235, ip=10.0.45.178) Loss: 5.733274459838867
(RayTrainWorker pid=417, ip=10.0.17.31) Loss: 5.9500956535339355


2023-03-07 16:23:53,988	WARNING tune.py:150 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-07 16:23:56,613	ERROR tune.py:821 -- Trials did not complete: [TorchTrainer_d9af1_00000]
2023-03-07 16:23:56,614	INFO tune.py:825 -- Total run time: 285.09 seconds (283.31 seconds for the tuning loop).
2023-03-07 16:23:56,614	WARNING tune.py:831 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:23:56,616	INFO utils.py:57 -- Worker 1 has failed.


In [7]:
import os
os.environ[
        "TORCH_DISTRIBUTED_DEBUG"
    ] = "DETAIL"

In [8]:
result = trainer.fit()

(RayTrainWorker pid=303, ip=10.0.5.7) 2023-03-07 15:09:46,491	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:46,610	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:49,880	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> TaskPoolMapOperator[MapBatches(<lambda>)]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:55,809	WARNING plan.py:528 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for mor

(autoscaler +26s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +26s) Adding 5 node(s) of type worker-node-type-0.


(TorchTrainer pid=3738, ip=10.0.48.48) /home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/dataset_iterator.py:64: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset/DatasetPipeline as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(TorchTrainer pid=3738, ip=10.0.48.48)   warnings.warn(
(RayTrainWorker pid=1124519) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=303, ip=10.0.5.7) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=1124519) wandb: wandb version 0.13.11 is available!  To upgrade, please run:
(RayTrainWorker pid=1124519) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=1124519) wandb: Tracking run with wandb version 0.13.10
(RayTrainWorker pid=1124519) w

(RayTrainWorker pid=1124519) CUDA AVAILABLE:  True
(RayTrainWorker pid=1124519) Applying spectral norm to text encoder LSTM


(RayTrainWorker pid=1124519) /tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/working_dir_files/_ray_pkg_eccc6979c79037b7d167507f1405cae4/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
(RayTrainWorker pid=1124519) The boolean parameter 'some' has been replaced with a string parameter 'mode'.
(RayTrainWorker pid=1124519) Q, R = torch.qr(A, some)
(RayTrainWorker pid=1124519) should be replaced with
(RayTrainWorker pid=1124519) Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
(RayTrainWorker pid=1124519)   W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a

...

(RayTrainWorker pid=303, ip=10.0.5.7) Loss: 66.65056610107422
(RayTrainWorker pid=1124519) Loss: 66.45819854736328
(autoscaler +6m24s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +6m28s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m34s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m38s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m44s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m48s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m54s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m59s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m4s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m8s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m14s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m18s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m24s) Rem

(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:821] [Rank 1] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=123, OpType=BROADCAST, Timeout(ms)=1800000) ran for 1800693 milliseconds before timing out.
(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:456] Some NCCL operations have failed or timed out. Due to the asynchronous nature of CUDA kernels, subsequent GPU operations might run on corrupted/incomplete data.
(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:461] To avoid data inconsistency, we are taking the entire process down.
(RayTrainWorker pid=1124519) [2023-03-07 15:42:17,362 E 1124519 1124595] logging.cc:97: Unhandled exception: St13runtime_error. what(): [Rank 1] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=123, OpType=BROADCAST, Timeout(ms)=1800000) ran for 1800693 milliseconds before timing out.
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weig

Trial name,date,hostname,node_ip,pid,timestamp,trial_id
TorchTrainer_209d8_00000,2023-03-07_15-09-43,ip-10-0-48-48,10.0.48.48,3738,1678230583,209d8_00000


2023-03-07 15:42:30,245	ERROR tune.py:821 -- Trials did not complete: [TorchTrainer_209d8_00000]
2023-03-07 15:42:30,246	INFO tune.py:825 -- Total run time: 1974.69 seconds (1962.16 seconds for the tuning loop).


RayTaskError: [36mray::_Inner.train()[39m (pid=3738, ip=10.0.48.48, repr=TorchTrainer)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 360, in train
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/base_trainer.py", line 737, in _trainable_func
    super()._trainable_func(self._merged_config, reporter, checkpoint_dir)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/base_trainer.py", line 647, in train_func
    trainer.training_loop()
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/data_parallel_trainer.py", line 433, in training_loop
  

In [9]:
??librosa_mel_fn

Signature:
librosa_mel_fn(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm='slaney',
    dtype=<class 'numpy.float32'>,
)
Source:   
@cache(level=10)
def mel(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm="slaney",
    dtype=np.float32,
):
    """Create a Mel filter-bank.

    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.

    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal

    n_fft     : int > 0 [scalar]
        number of FFT components

    n_mels    : int > 0 [scalar]
        number of Mel bands to generate

    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)

    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``

    htk       : bool [scalar]
        use HTK formula instead of Slaney

    norm : {None, 'slaney', or num

In [1]:
from librosa.filters import mel as librosa_mel_fn
help(librosa_mel_fn)

Help on function mel in module librosa.filters:

mel(sr, n_fft, n_mels=128, fmin=0.0, fmax=None, htk=False, norm='slaney', dtype=<class 'numpy.float32'>)
    Create a Mel filter-bank.
    
    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.
    
    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal
    
    n_fft     : int > 0 [scalar]
        number of FFT components
    
    n_mels    : int > 0 [scalar]
        number of Mel bands to generate
    
    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)
    
    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``
    
    htk       : bool [scalar]
        use HTK formula instead of Slaney
    
    norm : {None, 'slaney', or number} [scalar]
        If 'slaney', divide the triangular mel weights by the width of the mel band
        (area normalization).
    
      

In [1]:
import librosa
from librosa.util import pad_center
librosa.__version__

'0.10.0'

In [2]:
# import pad_center
help(pad_center)

Help on function pad_center in module librosa.util.utils:

pad_center(data: 'np.ndarray', *, size: 'int', axis: 'int' = -1, **kwargs: 'Any') -> 'np.ndarray'
    Pad an array to a target length along a target axis.
    
    This differs from `np.pad` by centering the data prior to padding,
    analogous to `str.center`
    
    Examples
    --------
    >>> # Generate a vector
    >>> data = np.ones(5)
    >>> librosa.util.pad_center(data, size=10, mode='constant')
    array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.])
    
    >>> # Pad a matrix along its first dimension
    >>> data = np.ones((3, 5))
    >>> librosa.util.pad_center(data, size=7, axis=0)
    array([[ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.]])
    >>> # Or its second dimension
    >>> librosa.util.pad_cent

In [3]:
import numpy as np
pad_center(np.ones(1000), 5000)

TypeError: pad_center() takes 1 positional argument but 2 were given

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install librosa==0.8.1

  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.11.1 requires cython==0.29.28, but you have cython 0.29.33 which is incompatible.
tts 0.11.1 requires inflect==5.6.0, but you have inflect 6.0.2 which is incompatible.
tts 0.11.1 requires librosa==0.8.0, but you have librosa 0.8.1 which is incompatible.
tts 0.11.1 requires numpy==1.22.4; python_version == "3.10", but you have numpy 1.22.0 which is incompatible.


In [2]:
import pandas as pd
import ray
parallelism_length = 400

In [3]:
lj_df = pd.read_csv(
    "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100_edited.txt",
    sep="|",
    header=None,
    quoting=3,
    names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
)
speaker_ids = lj_df.speaker_id.tolist()
speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
speaker_ids_ds = speaker_ids_ds.map_batches(
    lambda x: x, batch_format="pyarrow", batch_size=None
)

2023-03-06 16:54:26,264	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-06 16:54:26,291	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-06 16:54:26,316	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip' (6.75MiB) to Ray cluster...
2023-03-06 16:54:26,438	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip'.


In [4]:
lj_df

,path,transcript,speaker_id
0,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"Printing, in the only sense with which we are ...",0
1,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,in being comparatively modern.,0
2,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,For although the Chinese took impressions from...,0
3,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"produced the block books, which were the immed...",0
4,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,the invention of movable metal letters in the ...,0
...,...,...,...
95,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,have now come into general use and are obvious...,0
96,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,a little reduced in ugliness. The design of th...,0
97,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"and the whole effect is a little too gray, owi...",0
98,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"It must be remembered, however, that most mode...",0


In [3]:
output_dataset = speaker_ids_ds.map_batches(
    lambda table: table.rename(
        columns={

            "value_1": "speaker_id"
        }
    )
)

In [4]:
output_dataset.speaker_id.tolist()

AttributeError: 'Dataset' object has no attribute 'speaker_id'

(autoscaler +2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +2s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +7s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +12s) Removing 2 nodes of type worker-node-type-0 (idle).


In [ ]:
pd.DataFrame(output_dataset)

In [ ]:
output_dataset

2023-03-06 16:50:22,749	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(<lambda>)]
MapBatches(<lambda>)->MapBatches(<lambda>): 100%|██████████| 100/100 [00:03<00:00, 28.88it/s]


In [39]:
#     lj_df = pd.read_csv(
#         "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/vctk_mic1/all_with_embs.txt",
#         sep="|",
#         header=None,
#         quoting=3,
#         names=["path", "speaker_id", "transcript", "dataset_audio_file_id", "emb_path"],
#     )

In [38]:
# lj_df['path'][0]

In [37]:
# transcripts_ds = ray.data.from_items(transcripts, parallelism=parallelism_length)
# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# dataset_audio_file_ids = ray.data.from_items(
#     dataset_audio_files, parallelism=parallelism_length
# )

# pitch_paths_ds = ray.data.read_binary_files(
#     pitch_paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )

# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# transcripts_ds = transcripts_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# dataset_audio_file_ids = dataset_audio_file_ids.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)
# pitches_paths_ds = pitch_paths_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
